In [1]:
#-*- coding: utf-8 -*-
%matplotlib inline
from functools import reduce
import pandas as pd
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import shutil
import traceback
import itertools
import csv
import os
import glob
import math
import wave
import time as tm
import scipy.fftpack
from pylab import *
from scipy import stats
from sklearn import datasets
from sklearn.model_selection import train_test_split
# from sklearn import cross_validation

In [3]:
def notify_finishing(is_silent=True):
    if is_silent:
        title = '"report"'
        content = '"Finish"'
        cmd = "osascript -e 'display notification " + content +  " with title " + title + "'"
        os.system(cmd)
    else:
        cmd = "say Finish"
        os.system(cmd)

In [ ]:
##情報が欲しい距離のインデックスを取得
def d_index_set(d, index_num, width):
    dmin_index = []
    dmax_index = []
    for n in range(index_num):
        dmin_index.append(len(d[d < n * 0.8 - width]))
        dmax_index.append(len(d[d< n * 0.8 + width]))
    return dmin_index, dmax_index

In [2]:
#パラメータ設定
Fs = 1/ (8 * (pow(10, (-6)))) #サンプリング周波数
Tc = 1024*(10**(-6))#掃引時間
B = 187.4 * pow(10, 6) #掃引帯域幅
c = 3 * pow(10, 8) #光速
integral_range = 0.3 #距離のプラマイの積分範囲
Sa = 4096 #大体(データ数 / 測定時間)　

data_length = 102400
time_sum = 12.5
window = 128
overlap = 128
NFFT = pow(2, 16)
M = math.ceil((data_length - window)/overlap) - 1
win_func = hanning(M)
delta_f = Fs / NFFT
xaxis = delta_f * np.arange(NFFT)
tau = xaxis * Tc / B #Tc:B = tau:ビート周波数(xaxisに対応)
d = c * tau / 2 #距離推定

dmin_index, dmax_index = d_index_set(d, index_num = 6, width= 0.3)

In [4]:
class FileOperation:
    def __init__(self):
        pass

    def merge_csv(name, temp_dir, orginal_dir ):
    """a_1.txt, a_2.txt, a_3.txtのようなファイルを結合"""
        file_index = 1

        '''original_dirをtarget_dirにコピーしてそこの中で重複をなくす作業を行う'''
        target_dir = original_dir.rsplit('/', 1)[0] + '/' + temp_dir
        if os.path.exists(target_dir):
            shutil.rmtree(target_dir)
        shutil.copytree(original_dir, target_dir)

        for i in range(2):
            if i == 0:
                files = sorted(glob.glob(target_dir + '/' + name + '_*[a-z][1-9].txt'))# 1ケタついてる用
            elif i == 1:
                files = sorted(glob.glob(target_dir + '/' + name + '_*[a-z][1-9][0-9].txt'))# 2ケタついてる用

            for file in files:
                file_name = file.split('.')[0]
                file_name_short = file.rsplit('/', 1)[1].split('.')[0]
                print(file_name)
                same_files = sorted(glob.glob(file_name + '_' + '*' + '.txt'))
                if len(same_files) > 0:
                    for same_file in same_files:
                        csv1 = pd.read_csv(file_name + '.txt', delimiter="   ", header=None, skiprows=7, engine='python').iloc[:, 0:3]
                        csv2 = pd.read_csv(same_file, delimiter="   ", header=None, engine='python', skipfooter=1).iloc[:, 0:3]
                        csv_concat = pd.concat([csv1, csv2], ignore_index=True)
                        np.savetxt(file_name + '.txt', csv_concat, fmt=["%1.0d", "%0.3f", "%0.3f"], delimiter="   ")
                        os.remove(same_file)
    
    ##不必要なファイルの除外
    def set_file_excluded(files, exclude_actions):
        file_excluded = []
        for i, file in enumerate(files):
            action = file.rsplit('/', 1)[1].split('.')[0].split('_')[1][:-1]
            if action not in exclude_actions:
                file_excluded.append(file)
        return file_excluded
    
    ##不必要な行動変化の時間データの除外
    def set_source_excluded(source, exclude_actions):
        for i, action in enumerate(exclude_actions):
            source = source[(source["act"] != action)]
        source_excluded = source.reset_index(drop=True)
        return source_excluded
        
    def get_action(file):
        action = file.rsplit('/', 1)[1].split('.')[0].split('_')[1]
        return action
    
    def get_subject(file):
        subject = file.rsplit('/', 2)[1]
        return subject
            
    def label_from_action(action):
        if action == "fall" or action == "standfall":
            label = [1]
        elif action == "front":
            label = [2]
        elif action == "side":
            label = [3]
        elif action == "back":
            label = [4]
        elif action == "bodychange" or action == "other":
            label = [5]
        elif action == "sit":
            label = [6]
        elif action == "stand":
            label = [7]
        else:
            label = [0]
            print("Anything is wrong.")
        return label
    
    
    ##ラベル付けしたデータの保存
    def save_labeled_data(feature_with_label, savefolder, subject, action, source_index):
        if source_index < 10:
            source_index_str = "00" + str(source_index)
        elif source_index < 100:
            source_index_str= "0" + str(source_index)
        else:
            source_index_str = str(source_index)
        save_name = savefolder + subject + '_' + action +'_' + source_index_str + '.csv'
        np.savetxt(save_name, feature_with_label, delimiter=',')
        #csvのデータ数が１なら削除
        if os.path.getsize(save_name) == 1:
            os.remove(save_name)
    
    def merge_files(self, data_folder, save_folder):
        files = []
        files = sorted(glob.glob(data_folder))
        files_excluded = self.set_file_excluded(files, exclude_actions)
        for i, file in enumerate(files_excluded):
            action = get_action(file)
            subject = get_subject(file)

            doppler = self.doppler_from_IQdata(file)
            data_length = len(doppler)
            time_sum = data_length / Sa

            cdf = CalculateDistanceFeatures(doppler)
            spec = cdf.__spec_tdp_from_doppler()
            distance = cdf.__max_distance_from_spec(Spec)
            distance_reshaped = np.reshape(np.array(distance), ( len(distance), 1))

            if int(i / 100) >= 1:
                number = str(i)
            elif int(i / 10) >= 1:    
                number = "0"+str(i)
            else:
                number = "00"+str(i)
            with open(savefolder + subject + "_" + action + "_" + number  + '.csv', 'w') as csv_file:
                for index, content in enumerate(distance_reshaped):
                    writer = csv.writer(csv_file)
                    writer.writerow(content)                    
                    
    def sql_operation(self, data_path):
        import sqlite3
        from contextlib import closing
        files = []
        files = sorted(glob.glob(data_path))
        exclude_actions = ['no']
        files_excluded = set_file_excluded(files, exclude_actions)

        dbname = 'experiment.db'
        with closing(sqlite3.connect(dbname)) as conn:
            c = conn.cursor()
            drop_table = '''drop table if exists subjects'''
            conn.execute(drop_table)

        #         executeメソッドでSQL文を実行する
            create_table = '''create table subjects (id int primary key, name varchar(64), action varchar(64), start int, end int, std_after float, mean_after float, range_after float, grav_posi float, diff_sum float, std float, mean float, max float, min float, range float, mean_diff float, time float, label integer)'''    
            c.execute(create_table)
            for i, file in enumerate(files_excluded):
                name = file.rsplit(".")[0].rsplit("_")[5].split("/")[1]
                action = file.rsplit(".")[0].rsplit("_")[6]

            #カラムの追加
        #         add_column="alter table　subjects add column label integer;"
        #         add_column="ALTER TABLE subjects ADD COLUMN label integer"
        #         c.execute(add_column)
            # 受注表(csv)を開く。
                with open(file, 'r') as f: 
                    b = csv.reader(f)
                    header = next(b)
                    subject = tuple([name, action])
                    subject = tuple(tuple([str(i)]) + subject +  tuple(header))
        #             # tableに各行のデータを挿入する。
                    print(subject)
        # #                 cur.execute('INSERT INTO subjects　(id int primary key, std_after float, mean_after float, range_after float, grav_posi float, diff_sum float, std float, mean float, max float, min float, range float, mean_diff float, time float) values (?,?,?,?,?,?,?,?,?,?,?,?,?);', t)　
                    sql = 'insert into subjects (id, name, action, std_after, mean_after, range_after, grav_posi, diff_sum, std, mean, max, min, range, mean_diff, time, label) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
        #             sql = 'insert into subjects (name, action) values (?, ?)'
        #             sql = "UPDATE subjects SET name=name WHERE name='None'"
        #             sql = "replace into subjects(id, name, action) values(?, ?, ?);"
                    c.execute(sql, subject)
            conn.commit()
            conn.close()

In [5]:
class MakeLabelFile:
    def __init__(self, data_per_second, st_threshold, boundary, sec_start, sec_end, time_interval, convolve_range, observe_time):
        self.data_per_second = data_per_second
        self.st_threshold = st_threshold
        self.boundary = boundary
        self.sec_start = sec_start#秒後から見る
        self.sec_end = sec_end #秒前まで見る
        self.starts, self.ends, self.numbers, self.actions, self.states = [], [], [], [], []
        self.time_interval = time_interval
        self.convolve_box = np.ones(convolve_range)/convolve_range
        self.observe_time = observe_time


    def __judge_label(self, std_start, std_end):
        if std_start < self.label_start:
            start_detected = self.label_start
        elif self.label_start <= std_start and std_start <= self.label_end:
            start_detected = std_start
        else:
            start_detected = self.label_end

        if std_end < self.label_start:
            end_detected = self.label_start
        elif self.label_start <= std_end and std_end <= self.label_end:
            end_detected = std_end
        else:
            end_detected = self.label_end    

        if (end_detected - start_detected)/(self.label_end - self.label_start) >= self.boundary:
            label = FileOperation.label_from_action(self.action)#actionにより1~7のlabelを返す
        else:
            label = [8]

        return label


    def __extend_datas(self):
        start_flag, end_flag = 0, 0
        called_flag = 0
        data_interval = int(self.data_per_second * self.time_interval)
        for data_i in range(self.data_per_second*self.sec_start, len(self.distance) - self.data_per_second * self.sec_end, data_interval):
            if np.std(self.distance[data_i:data_i+data_interval]) >self.st_threshold:
                if start_flag == 0:
                    start_flag = 1
                    self.starts.extend([data_i/self.data_per_second])
                    self.numbers.extend([self.file_index])
                    self.actions.append(self.action)
                    if (called_flag == 1) and (self.starts[-1] - self.ends[-1] < self.observe_time):
                        del self.starts[-2], self.numbers[-2], self.actions[-2], self.ends[-1], self.states[-1]
            else:
                if start_flag == 1:
                    start_flag = 0
                    called_flag = 1
                    self.ends.extend([data_i/self.data_per_second])
                    label = self.__judge_label(std_start = self.starts[-1], std_end = self.ends[-1])
                    self.states.append(label)

        if len(self.ends) != len(self.starts):
            self.ends.extend([data_i/self.data_per_second])
            label = self.__judge_label(std_start = self.starts[-1], std_end = self.ends[-1])
            self.states.append(label)
        
        
    def make_label_file(self, data_folder, time_file, exclude_actions, save_file):
        files = []
        files = sorted(glob.glob(data_folder))
        source = pd.read_csv(time_file, delimiter=',', encoding='cp932', engine='python')
        files_excluded = FileOperation.set_file_excluded(files, exclude_actions)
        source_excluded = FileOperation.set_source_excluded(source, exclude_actions)
        labels_start, labels_end = source_excluded["change_start_s"], source_excluded["change_end_s"]
        
        for file_index, file in enumerate(files_excluded):
            self.file_index = file_index
            self.action = FileOperation.get_action(file)[:-1]
            self.distance_original = pd.read_csv(file, skiprows= 0, header=None, names=['distance'], engine='python')['distance']
            self.distance = np.convolve(self.distance_original, self.convolve_box, 'same')
            self.label_start, self.label_end = labels_start[file_index], labels_end[file_index]
            self.__extend_datas()
            
#         all_action = reduce(lambda x,y: x+y, self.actions)
        data_extended = pd.DataFrame({"number":self.numbers, "change_start_s":self.starts, "change_end_s":self.ends, "state":self.states, "act": self.actions})
        data_extended.to_csv(save_file)

In [14]:
class MakeFeaturesFile:
    def __init__(self, see_after_time, see_before_time, time_diff, t_observe, t_division):
        self.see_after_time=see_after_time
        self.see_before_time=see_before_time
        self.time_diff = time_diff
        self.t_observe = t_observe
        self.t_division = t_division
        
    def _doppler_from_IQdata(self, file):
        # 空の配列にIQ信号を入れる．要素がないところには0を入れる
        try:
            temp = pd.read_csv(file, delimiter="   ", skiprows= 0, header=None, names=['No.', 'I','Q'], engine='python')
            zero = np.zeros((temp.shape[0], 2), dtype=np.int64)
            data_length = temp.shape[0]
            IQdata = pd.DataFrame(zero, columns=['I','Q'])
            IQdata.I = temp.I
            IQdata.Q = temp.Q
            doppler = IQdata.I+1j* IQdata.Q
        except:
            temp = pd.read_csv(file, delimiter="   ", skiprows= 7, header=None, names=['No.', 'I','Q'], engine='python')
            zero = np.zeros((temp.shape[0], 2), dtype=np.int64)
            data_length = temp.shape[0]
            IQdata = pd.DataFrame(zero, columns=['I','Q'])
            IQdata.I = temp.I
            IQdata.Q = temp.Q
            doppler = IQdata.I+1j* IQdata.Q

        self.data_length = len(doppler)
        self.time_sum = self.data_length / Sa
        self.feature_bin = int(self.time_sum * Sa / overlap) #Specのnp.shapeと少しずれあり
        return doppler
    
    #time, distance, powerのspectrogram
    def _spec_tdp_from_doppler(self, doppler):
        self.M = math.ceil((self.data_length-window)/overlap)  - 1
#         new_x = zeros(window + (self.M + 1) * window, dtype = complex64)
        new_x = zeros(window + (self.M + 1) * overlap, dtype = complex64)
        new_x[: self.data_length] = doppler # 信号をいい感じの長さにする
        Spec = zeros([self.M, NFFT], dtype = complex64) # スペクトログラムの初期化(複素数型)
        win_func = hanning(window)
        #STFTをする
        for m in range(self.M):
            start = overlap * m
            Spec[m, :] = abs(fft(new_x[start : start + window] * win_func, NFFT))**2
        Spec = abs(Spec)
        
        Spec_in_drange = Spec[0:, dmin_index[0]:dmax_index[3]]
        return Spec_in_drange
    
    def _spec_bg_cut(self, Spec, Spec_bg):
        spec_len = np.shape(Spec)[0]
        spec_bg_len = np.shape(Spec_bg)[0]
        if spec_len <= spec_bg_len:
            Spec_bg = Spec_bg[0:spec_len, :]
        else:
            Spec_bg = np.concatenate([Spec_bg, Spec_bg[0:spec_len-spec_bg_len, :]], axis=0)
        
        Spec_bg_cut = abs(Spec - Spec_bg)
        return Spec_bg_cut
    
    def _max_distance_from_spec(self, Spec):
        distance = d[np.argmax(Spec, axis=1)]
        return distance
    
    def _gra_med_from_spec(self, Spec, label_start, label_end):
        spec_len = np.shape(Spec)[0]
        change_startframe = int(label_start*spec_len/ self.time_sum)
        change_endframe = int(label_end*spec_len/ self.time_sum)
        gra_med = [np.sum(d[dmin_index[0]:dmax_index[3]] * np.sum(abs(Spec[change_startframe:change_endframe, dmin_index[0]:dmax_index[3]]), axis=0))/np.sum(abs(Spec[change_startframe:change_endframe, dmin_index[0]:dmax_index[3]]))]    
        return gra_med
    
    def _create_features_labeled(self, Spec, distance, label, label_start, label_end):
        gra_med = self._gra_med_from_spec(Spec, label_start, label_end)
        #距離の特徴量にSpecから取ったpowerの時系列から特徴量取得して連結
        max_power = np.max(Spec, axis=1)
        features_with_label = self._feature_from_distance(distance, label_start,  label_end, [label], gra_med)
        min_power = np.min(Spec, axis=1)
        mean_power = np.mean(Spec, axis=1)
        std_power = np.std(Spec, axis=1)
        
        min_distance = d[np.argmin(Spec, axis=1)]
        features_of_min_distance = self._feature_from_distance(min_distance, label_start,  label_end, [label], gra_med)
        
        features_of_max_power = self._feature_from_distance(max_power, label_start,  label_end, [label], gra_med)
        features_of_min_power = self._feature_from_distance(min_power, label_start,  label_end, [label], gra_med)
        features_of_mean_power = self._feature_from_distance(mean_power, label_start,  label_end, [label], gra_med)
        features_of_std_power = self._feature_from_distance(std_power, label_start,  label_end, [label], gra_med)
        
        features_with_label = np.concatenate([features_with_label[0][:-1], features_of_min_distance[0][:-1], features_of_max_power[0][:-1], features_of_min_power[0][:-1], features_of_mean_power[0][:-1], features_of_std_power[0]])
        features_with_label = np.array(features_with_label).reshape(1, len(features_with_label))#形整えた
        return features_with_label
    
    def _create_features_distance(self, Spec, distance, label, label_start, label_end):
        gra_med = self._gra_med_from_spec(Spec, label_start, label_end)
        features_with_label = self._feature_from_distance(distance, label_start,  label_end, [label], gra_med)
        features_with_label = np.array([features_with_label[0][1], features_with_label[0][-1]])
        features_with_label = np.array(features_with_label).reshape(1, len(features_with_label))#形整えた
        return features_with_label
        
    def _create_features_grad_std(self, Spec, label, label_start, label_end):
        gra_med = self._gra_med_from_spec(Spec, label_start, label_end)
        std_power = np.std(abs(Spec), axis=1)
        mean_std_power = np.mean(std_power)
        features_with_label = np.array([round(gra_med[0], 3), mean_std_power, label])
        features_with_label = np.array(features_with_label).reshape(1, len(features_with_label))#形整えた
        return features_with_label
        
    def make_distance_file(self, data_folder, exclude_actions, save_folder):
        files = sorted(glob.glob(data_folder))
        files_excluded = FileOperation.set_file_excluded(files, exclude_actions)
        
        for file_index, file in enumerate(files_excluded):        
            doppler = self. _doppler_from_IQdata(file)
            Spec = self._spec_tdp_from_doppler(doppler)
            distance = self._max_distance_from_spec(Spec)
            action, subject = FileOperation.get_action(file), FileOperation.get_subject(file)
            FileOperation.save_labeled_data(distance, save_folder, subject, action, file_index)
    
    def _feature_from_distance(self, distance, label_start, label_end, label, gra_med):
        change_startframe = int(label_start*len(distance)/ self.time_sum)
        change_endframe = int(label_end*len(distance)/ self.time_sum)
        
        time_after = self.see_after_time 
        after_frame =  int(self.see_after_time*len(distance)/ self.time_sum)
    
        dist_after = distance[change_endframe:change_endframe+after_frame]
        dist_middle = distance[change_startframe:change_endframe+1]
    
        std_after = [np.std(dist_after)] #-3. 変化終了後の距離の標準偏差
        mean_after = [np.mean(dist_after)] #-2. 変化終了後の距離平均
        gra_med = gra_med  #3. 重心の位置
        d_diff = zeros(change_endframe - change_startframe, dtype=float64)
        d_diff[:len(dist_middle)-1] = np.diff(dist_middle, n=1)
        sum_abs_dist = [np.sum(list(map(lambda x: abs(x), d_diff)))] #4. 前後の距離の差の絶対値の合計
        diff_std = [np.std(d_diff)] #4.3 前後の差の準偏偏差
#             features.extend([np.sum(np.diff(distance[j], n=2))]) #4.6前後の差の差の和
        
        _diff_jump = int(self.time_diff*27)
        diff_broad_mean = [np.mean(dist_middle[-1:0:-_diff_jump])]
        diff_broad_std = [np.std(dist_middle[-1:0:-_diff_jump])]
        
        #t_divisionずつで区切った場合のベクトル
        _div_std = []
        _div_mean = []       
        for i in range(int(self.t_observe/self.t_division)*2-1):
            div_distance = distance[change_endframe-int((i*0.5)*27*self.t_division):change_endframe-int((i*0.5+1)*27*self.t_division):-1]
            _div_std.append(np.std(div_distance))
            _div_mean.append(np.mean(div_distance))
        div_feature = np.concatenate([_div_std, _div_mean])
        
        dis_std = [np.std(dist_middle)] #5. 距離の標準偏差
        dis_mean = [np.mean(dist_middle)] #6. 距離の平均値
        dis_max = [np.max(dist_middle)] #8. 距離の最大値
        dis_min = [np.min(dist_middle)]#9. 距離の最小値
        diff_max_min = [dis_max[0] - dis_min[0]]#10. 距離の最大値と最小値の差
        diff_mean = [dis_mean[0] - mean_after[0]]#11. 行動後の平均距離 -  行動中の平均距離
        action_time = [label_end - label_start]#12. 行動時間
    
        features_with_label = []
        features_with_label.extend(np.concatenate([std_after, mean_after, gra_med, sum_abs_dist, diff_std, diff_broad_mean, diff_broad_std, div_feature, dis_std, dis_mean, dis_max, diff_max_min, diff_mean, action_time, label]))
        features_with_label_reshaped = np.array(features_with_label).reshape(1, len(features_with_label))
        return features_with_label_reshaped

    def make_features_files(self, data_folder, time_file, exclude_actions, save_folder, method, should_bg_cut):
        files = sorted(glob.glob(data_folder))
        files_excluded = FileOperation.set_file_excluded(files, exclude_actions)
        source = pd.read_csv(time_file, delimiter=',', encoding='cp932', engine='python')
        source_excluded = FileOperation.set_source_excluded(source, exclude_actions)
        labels_start, labels_end, numbers, labels = source_excluded["change_start_s"], source_excluded["change_end_s"], source_excluded["number"], source_excluded["state"]
        source_index = 0
        doppler_bg = self. _doppler_from_IQdata(bg_path)
        Spec_bg = self._spec_tdp_from_doppler(doppler_bg)
        
        for file_index, file in enumerate(files_excluded):
            doppler = self. _doppler_from_IQdata(file)
            Spec = self._spec_tdp_from_doppler(doppler)
            if should_bg_cut:
                Spec_bg_cut = self._spec_bg_cut(Spec, Spec_bg)
            else:
                Spec_bg_cut = Spec
            distance = self._max_distance_from_spec(Spec_bg_cut) #従来法では必要なので残す
            action, subject = FileOperation.get_action(file), FileOperation.get_subject(file)
            
            while source_index < len(numbers) and numbers[source_index] == file_index:
                label_start, label_end = labels_start[source_index], labels_end[source_index]
                if label_end - label_start < 0 or label_start > self.time_sum :
                    source_index = source_index+1
                    continue
                label = float(labels[source_index][1])

                if method == "propose":
                    features_with_label = self._create_features_labeled(Spec, distance, label, label_start, label_end)
                elif method == "conv_dist":
                    features_with_label = self._create_features_distance(Spec_bg_cut, distance, label, label_start, label_end)
                elif method == "conv_grad_std":
                    features_with_label = self._create_features_grad_std(Spec_bg_cut, label, label_start, label_end)
                FileOperation.save_labeled_data(features_with_label, save_folder, subject, action, source_index)

                source_index = source_index+1